# Análise de Dados com Python

### Desafio:

A empresa XPTO fornece vários serviços, entre os quais: televisao e internet

O problema é que, de acordo com o historico dos ultimos anos, a empresa está com uma perda que ronda os 26% dos clientes.

Isso representa uma perda de milhões para a empresa.

O que a empresa precisa fazer para resolver a situação?

Base de Dados: https://drive.google.com/file/d/1pJbGLu5WZxVvL9FzFHqrwW2NI-WQ3TN6/view?usp=sharing


In [1]:
import pandas as pd
#import plotly.express as px
tabela = pd.read_csv('xpto_users.csv')
display(tabela)

Unnamed: 0   IDCliente     Genero  Reformado Casado Dependentes  \
0           1869  7010-BRBUU  Masculino          0    Sim         Sim   
1           4528  9688-YGXVR   Feminino          0    Nao         Nao   
2           6344  9286-DOJGF   Feminino          1    Sim         Nao   
3           6739  6994-KERXL  Masculino          0    Nao         Nao   
4            432  2181-UAESM  Masculino          0    Nao         Nao   
...          ...         ...        ...        ...    ...         ...   
5981        3772  0684-AOSIH  Masculino          0    Sim         Nao   
5982        5191  5982-PSMKW   Feminino          0    Sim         Sim   
5983        5226  8044-BGWPI  Masculino          0    Sim         Sim   
5984        5390  7450-NWRTR  Masculino          1    Nao         Nao   
5985         860  4795-UXVCJ  Masculino          0    Nao         Nao   

      MesesComoCliente ServicoTelefone MultiplasLinhas ServicoInternet  ...  \
0                   72             Sim             Sim             Nao  ...   
1                   44             Sim             Nao           Fibra  ...   
2                   38             Sim             Sim           Fibra  ...   
3                    4             Sim             Nao             DSL  ...   
4                    2             Sim             Nao             DSL  ...   
...                ...             ...             ...             ...  ...   
5981                 1             Sim             Nao           Fibra  ...   
5982                23             Sim             Sim             DSL  ...   
5983                12             Sim             Nao             Nao  ...   
5984                12             Sim             Sim           Fibra  ...   
5985                26             Sim             Nao             Nao  ...   

     ServicoSuporteTecnico ServicoStreamingTV ServicoFilmes TipoContrato  \
0              SemInternet        SemInternet   SemInternet       2 anos   
1                      Nao                Sim           Nao       Mensal   
2                      Nao                Nao           Nao       Mensal   
3                      Nao                Nao           Sim       Mensal   
4                      Nao                Nao           Nao       Mensal   
...                    ...                ...           ...          ...   
5981                   Nao                Sim           Sim       Mensal   
5982                   Sim                Sim           Sim       2 anos   
5983           SemInternet        SemInternet   SemInternet       Mensal   
5984                   Nao                Sim           Sim       Mensal   
5985           SemInternet        SemInternet   SemInternet        Anual   

     FaturaDigital    FormaPagamento ValorMensal TotalGasto Churn  Codigo  
0              Nao     CartaoCredito       24.10    1734.65   Nao     NaN  
1              Sim     CartaoCredito       88.15     3973.2   Nao     NaN  
2              Sim  DebitoAutomatico       74.95    2869.85   Sim     NaN  
3              Sim  BoletoEletronico       55.90      238.5   Nao     NaN  
4              Nao  BoletoEletronico       53.45      119.5   Nao     NaN  
...            ...               ...         ...        ...   ...     ...  
5981           Sim  BoletoEletronico       95.00         95   Sim     NaN  
5982           Sim     CartaoCredito       91.10     2198.3   Nao     NaN  
5983           Sim  BoletoEletronico       21.15     306.05   Nao     NaN  
5984           Sim  BoletoEletronico       99.45    1200.15   Sim     NaN  
5985           Nao     CartaoCredito       19.80      457.3   Nao     NaN  

[5986 rows x 23 columns]

In [2]:
#
# Passo 3: Tratamento de dados
# - Valores que são reconhecidos erradamente
tabela["TotalGasto"] = pd.to_numeric(tabela["TotalGasto"], errors="coerce")

# - Valores vazios
# Eliminar as colunas vazias
# axis = 0 _> linha ou axis = 1 _> coluna
tabela = tabela.dropna(how="all", axis=1)
# Eliminar as linhas vazias
tabela = tabela.dropna(how="any", axis=0)

print(tabela.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5974 entries, 0 to 5985
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5974 non-null   int64  
 1   IDCliente               5974 non-null   object 
 2   Genero                  5974 non-null   object 
 3   Reformado               5974 non-null   int64  
 4   Casado                  5974 non-null   object 
 5   Dependentes             5974 non-null   object 
 6   MesesComoCliente        5974 non-null   int64  
 7   ServicoTelefone         5974 non-null   object 
 8   MultiplasLinhas         5974 non-null   object 
 9   ServicoInternet         5974 non-null   object 
 10  ServicoSegurancaOnline  5974 non-null   object 
 11  ServicoBackupOnline     5974 non-null   object 
 12  ProtecaoEquipamento     5974 non-null   object 
 13  ServicoSuporteTecnico   5974 non-null   object 
 14  ServicoStreamingTV      5974 non-null   

In [3]:
# Passo 4: Análise Inicial
# percentagem das pessoas que cancelam?
print(tabela["Churn"].value_counts())
print(tabela["Churn"].value_counts(normalize=True).map("{:.1%}".format))

Nao    4387
Sim    1587
Name: Churn, dtype: int64
Nao    73.4%
Sim    26.6%
Name: Churn, dtype: object


In [4]:
# Passo 5: Análise Mais completa
# Comparar cada coluna da nossa tabela com a coluna de cancelamento
import plotly.express as px


# etapa 1: Criar o gráfico
for coluna in tabela.columns:
    # para edições nos gráficos: https://plotly.com/python/histograms/
    # para mudar a cor do gráfico , color_discrete_sequence=["blue", "green"]
    grafico = px.histogram(tabela, x=coluna, color="Churn")
    # etapa 2: exibição do gráfico
    grafico.show()

In [ ]:
%pip install nbformat

### Conclusões e Ações

Escreva aqui suas conclusões:

- Clientes com contrato mensal tem mais hipotese de cancelar:
    - Podemos fazer promoções para o cliente ir para o contrato anual;
    
- Familias mais numerosas tendem a cancelar menos do que famílias com menos elementos
    - Podemos fazer promoções para as pessoas contratarem uma linha adicional de telefone
    
- MesesComoCliente baixos tem MUITO cancelamento. Clientes com pouco tempo como cliente tendem a cancelar muito
    - A primeira experiência do cliente na operadora pode ser ruim
    - Talvez a captação de clientes tá trazendo clientes desqualificados
    - Ideia: a gente pode criar incentivo pro cara ficar mais tempo como cliente
    
- QUanto mais serviços o cara tem, menos chance dele cancelar
    - podemos fazer promoções com mais serviços pro cliente
    
- Tem alguma coisa no nosso serviço de Fibra que tá fazendo os clientes cancelarem
    - Agir sobre a fibra
    
- Clientes no boleto tem MUITO mais chance de cancelar, então temos que fazer alguma ação para eles irem para as outras formas de pagamento